In [2]:
import qutip as qt
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib.gridspec import GridSpec

In [3]:
N = 6
J = 1.0
hx = 0.0
J_p_list = np.linspace(J, 0.0, 50)
OUTPATH = f"../data/tfim/tfim_spectrum_and_states_N{N}_hx{hx}_J1.0_Jp_1to0.npz"


def H_tfim_with_cut(N, J, J_p, hx):
    cut_pos = N // 2 - 1
    sx, sz = qt.sigmax(), qt.sigmaz()
    I2 = qt.qeye(2)

    def op_at(i, op):
        return qt.tensor([op if k == i else I2 for k  in range(N)])

    Sx = [op_at(i, sx) for i in range(N)]
    Sz = [op_at(i, sz) for i in range(N)]

    def Hex(i):
        return Sz[i] * Sz[i+1]

    H_cut = Hex(cut_pos)
    H_ex_rest = sum(Hex(i) for i in range(N-1) if i != cut_pos)
    H_trans = sum(Sx[i] for i in range(N))

    return - J * H_ex_rest - J_p * H_cut - hx * H_trans

dim = 2 ** N
E_all = np.empty((len(J_p_list), dim), dtype=float)
Psi_all = np.empty((len(J_p_list), dim, dim), dtype=complex)

for j, Jp in enumerate(J_p_list):
    H = H_tfim_with_cut(N, J, Jp, hx)
    evals, estates = H.eigenstates(sparse=False, sort='low')
    E_all[j] = np.real_if_close(evals)
    Psi_all[j] = np.stack([st.full().ravel() for st in estates], axis=0)


# left_side = list(range(N//2))
# psi_0 = qt.Qobj(Psi_all[-1][0], dims=[[2]*N, [1]])
# psi_3 = qt.Qobj(Psi_all[-1][3], dims=[[2]*N, [1]])
# print("0, 0 : ", qt.ptrace(psi_0 * psi_0.dag(), left_side))
# print((psi_0 * psi_0.dag()).tr())
# print("3, 3 : ", qt.ptrace(psi_3 * psi_3.dag(), left_side))
# print("0, 3 : ", qt.ptrace(psi_0 * psi_3.dag(), left_side))
# psi = psi_0 * np.sqrt(0.5) + psi_3 * np.sqrt(0.5)
# rho = psi * psi.dag()
# rho_A = qt.ptrace(rho, left_side)
# print((rho_A*rho_A).tr())


# for i in range(5):
#     print(rf"energie is {E_all[0][i]}, state is {Psi_all[0][i]}")
np.savez(OUTPATH, J_p=J_p_list, energies=E_all, eigvecs=Psi_all)
print(f"saved → {OUTPATH}")

saved → ../data/tfim/tfim_spectrum_and_states_N6_hx0.0_J1.0_Jp_1to0.npz
